In [1]:
import numpy as np
import pandas as pd
import datetime
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

import matplotlib.pyplot as plt
%matplotlib inline

import sys, os, gc, types
import time
from subprocess import check_output

import sklearn
from sklearn.model_selection import train_test_split

In [5]:
root_paths = [
    "/Users/jiayou/Dropbox/JuanCode/Insight/Project/data2/", #Jun Mac
    "/Users/jiayou/Dropbox/Documents/JuanCode/Insight/Project/data2/", #You Mini
    '/Users/junxie/Dropbox/JuanCode/Insight/Project/', #Pro
    '/mnt/WD Black/Dropbox/JuanCode/Insight/Project/data2/', #1080
    '/home/paperspace/Documents/insight/insight/data2/', #paperspace
    
]
root = None
for p in root_paths:
    if os.path.exists(p):
        root = p
        break
print(root)

/Users/junxie/Dropbox/JuanCode/Insight/Project/


In [21]:
prediction = {}
prediction['gb'] = pd.read_csv(root + 'data2/submission/test_prediction.gb_v2_r1.csv')
prediction['rnn'] = pd.read_csv(root + 'data2/submission/test_prediction.nn_v1.csv')
prediction['wave'] = pd.read_csv(root + 'data2/submission/test_prediction.wavenet_v1.csv')

In [ ]:
true = pd.read_csv(root + 'data2/solution_11_15.csv')
true.sort_values(by='Id', inplace=True)

In [41]:
true.reset_index(drop=True, inplace=True)

In [42]:
for name in ['gb', 'rnn', 'wave']:
    prediction[name].sort_values(by='Id', inplace=True)
    prediction[name].reset_index(drop=True, inplace=True)
    print((prediction[name].Id == true.Id).sum())
    true[name] = prediction[name].Visits

8993906
8993906
8993906


In [43]:
true.head()

,Id,Visits,Usage,gb,rnn,wave
0,0000009abb1c,150.0,Public,99.0,110.0,73
1,0000023b7434,4.0,Public,8.0,8.0,8
2,000002ddbe6e,163.0,Public,90.0,111.0,99
3,0000032f82ac,1129.0,Public,730.0,841.0,897
4,000003e266a3,87.0,Public,47.0,58.0,49


In [44]:
final_df = true[true.Usage == 'Public']

In [51]:
def calc_smape(y_true, y_pred):
    loss = np.abs(y_true - y_pred) / np.maximum(1e-6, (np.abs(y_true) + np.abs(y_pred))) * 200
    return loss

In [52]:
for name in ['gb', 'rnn', 'wave']:
    final_df['{}_SMAPE'.format(name)] = calc_smape(final_df.Visits, final_df[name])

/Users/junxie/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [59]:
for name in ['gb', 'rnn', 'wave']:
    print(name, final_df['{}_SMAPE'.format(name)].mean())

gb 39.54901135260361
rnn 38.60387041031282
wave 37.18357135298225


In [54]:
(final_df.gb_SMAPE - final_df.rnn_SMAPE).mean()

0.9451409422668224

In [62]:
(final_df.gb_SMAPE - final_df.rnn_SMAPE).std() / np.sqrt(145063)

0.06408154817332896

In [57]:
(final_df.rnn_SMAPE - final_df.wave_SMAPE).mean()

1.420299057298783

In [63]:
(final_df.rnn_SMAPE - final_df.wave_SMAPE).std() / np.sqrt(145063)

0.0723559644258328

In [61]:
len(final_df)

8752162